In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import gala.potential as gp
from gala.units import galactic
from gala.potential.scf import compute_coeffs, compute_coeffs_discrete

import schwimmbad

In [2]:
data = np.loadtxt('../../MW_anisotropy/code/test_snaps/LMC3_507K_part_b1_091.txt')

In [10]:
pos = data[:10000,0:3]

In [11]:
pos[0]

array([ -0.46587896, -31.65701675, -34.16439438])

In [12]:
mass = data[:,-1]
lmax = 10
r_s = 20
n = np.arange(0, 11, 1)

In [13]:
def parallel_coeffs(n):
    """
    parallelt comutation of the coefficients
    """
    
    S, T = compute_coeffs_discrete(pos, mass, n, lmax, r_s)
    return S, T


In [14]:
from schwimmbad import MPIPool
pool = MPIPool()


ValueError: Tried to create an MPI pool, but there was only one MPI process available. Need at least two.

In [ ]:
if not pool.is_master():
    pool.wait()
    sys.exit(0)

main(pool)

In [15]:
from schwimmbad import MultiPool


In [16]:
with MultiPool() as pool:
    values = list(pool.map(parallel_coeffs, n))

In [31]:
np.shape(values)

(11, 2)

In [33]:
np.shape(values[0])

(2, 1, 11, 11)

In [22]:
np.shape(values[1])

(2, 2, 11, 11)

In [35]:
np.shape(values[2][0])

(3, 11, 11)

In [36]:
compute_coeffs_discrete?

Signature: compute_coeffs_discrete(xyz, mass, nmax, lmax, r_s, skip_odd=False, skip_even=False, skip_m=False, compute_var=False)
Docstring:
Compute the expansion coefficients for representing the density distribution of input points
as a basis function expansion. The points, ``xyz``, are assumed to be samples from the
density distribution.

Computing the coefficients involves computing triple integrals which are computationally
expensive. For an example of how to parallelize the computation of the coefficients, see
``examples/parallel_compute_Anlm.py``.

Parameters
----------
xyz : array_like
    Samples from the density distribution. Should have shape ``(n_samples,3)``.
mass : array_like
    Mass of each sample. Should have shape ``(n_samples,)``.
nmax : int
    Maximum value of ``n`` for the radial expansion.
lmax : int
    Maximum value of ``l`` for the spherical harmonics.
r_s : numeric
    Scale radius.
skip_odd : bool (optional)
    Skip the odd terms in the angular portion of th